# defautling

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import scvi
import scanpy as sc
import anndata



In [ ]:
scale_factor=1e4

In [4]:
sc_data=sc.read_h5ad("/data/kjc2/projects/P330.CSA/rds/core_atlas-adata.h5ad")

# subsampling

In [7]:
down_size = 10  # 희석 배수
random_idx = np.random.choice(sc_data.n_obs, int(sc_data.n_obs/down_size), replace=False)
sc_data_sub = sc_data[random_idx].copy()
sc_data_sub.write_h5ad("/data/kjc2/projects/P330.CSA/rds/core_atlas-adata_down_10.h5ad")

down_size=100
random_idx = np.random.choice(sc_data.n_obs, int(sc_data.n_obs/down_size), replace=False)
sc_data_sub = sc_data[random_idx].copy()
sc_data_sub.write_h5ad("/data/kjc2/projects/P330.CSA/rds/core_atlas-adata_down_100.h5ad")

#del(sc_data)

# loading

In [3]:
sc_data_sub=sc.read_h5ad("/data/kjc2/projects/P330.CSA/rds/core_atlas-adata_down_100.h5ad")

# scVI

In [ ]:
scvi.model.SCVI.setup_anndata(sc_data_sub, batch_key="dataset",layer="counts")
# 모델 생성
model = scvi.model.SCVI(sc_data_sub)

# 모델 훈련 (GPU 사용 가능시 자동으로 활용)
model.train()
# Error displaying widget: model not found 무시해도 되는 오류라고 함.
# /home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/scvi/module/_vae.py:569: UserWarning: The value argument must be within the support of the distribution
# reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
# 이 오류는 raw counts(음이아닌정수)가 아닌 데이터가 들어온 것을 암시하며, 문제가 있다는 의미임.

# 잠재 공간 표현 얻기 (기본적으로 10차원)
adata.obsm["X_scVI_kjc"] = model.get_latent_representation()

# 정규화된 발현 값 얻기
# library_size='auto'는 각 세포의 라이브러리 크기를 고려하여 스케일링합니다.
# n_samples=1은 각 세포 및 유전자에 대해 하나의 값을 샘플링합니다 (기본값).
# 더 안정적인 값을 원하면 n_samples를 늘릴 수 있지만 계산량이 증가합니다.
sc_data_sub.layers["scvi_normalized"] = model.get_normalized_expression(
    library_size="auto",
    n_samples=1 # 필요에 따라 조절
)

# 결과 확인 (예: 특정 유전자 확인)
# print(adata.layers["scvi_normalized"][:5, :5])

/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/scvi/data/fields/_dataframe_field.py:187: UserWarning: Category 1 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jaecheon/anaconda3/envs/scenvi/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=111` in the `DataLoader` to improve performance.


Training:   0%|          | 0/188 [00:00<?, ?it/s]

In [18]:
sc_data_sub

AnnData object with n_obs × n_vars = 42649 × 28476
    obs: 'dataset', 'medical_condition', 'cancer_type', 'sample_id', 'sample_type', 'tumor_source', 'replicate', 'sample_tissue', 'anatomic_region', 'anatomic_location', 'tumor_stage', 'tumor_stage_TNM', 'tumor_stage_TNM_T', 'tumor_stage_TNM_N', 'tumor_stage_TNM_M', 'tumor_size', 'tumor_dimensions', 'tumor_grade', 'histological_type', 'microsatellite_status', 'mismatch_repair_deficiency_status', 'MLH1_promoter_methylation_status', 'MLH1_status', 'KRAS_status', 'BRAF_status', 'APC_status', 'TP53_status', 'PIK3CA_status', 'SMAD4_status', 'NRAS_status', 'MSH6_status', 'FBXW7_status', 'NOTCH1_status', 'MSH2_status', 'PMS2_status', 'POLE_status', 'ERBB2_status', 'STK11_status', 'HER2_status', 'CTNNB1_status', 'BRAS_status', 'patient_id', 'sex', 'age', 'treatment_status_before_resection', 'treatment_drug', 'treatment_response', 'RECIST', 'platform', 'platform_fine', 'cellranger_version', 'reference_genome', 'matrix_type', 'enrichment_cell_ty

In [ ]:
sc_data_sub

In [ ]:
sc_data_sub.write_h5ad("/data/kjc2/projects/P330.CSA/rds/core_atlas-adata_down_100_scVIed.h5ad")